In [16]:
from pyspark import SparkConf, SparkContext

In [17]:
conf = SparkConf().setMaster("local").setAppName("bus-stop")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=bus, master=local) created by getOrCreate at /var/folders/bp/0rpm363x6v5_d67px2lrg02w0000gn/T/ipykernel_1639/3917289345.py:3 

In [ ]:
import os
from pathlib import Path

directory = f"{Path(os.getcwd()).resolve().parent}/data"
csv_file = "bus.csv"

data = sc.textFile(f"file:///{directory}/{csv_file}")
header = data.first()
filetred_lines = data.filter(lambda row: row != header)
lines_split = filetred_lines.map(lambda x: x.split(",")[-1])

In [ ]:
count = lines_split.countByValue()

In [ ]:
import pandas as pd 
pd.Series(count).to_csv("city_count.csv", header=None)

In [ ]:
import matplotlib.pyplot as plt 

data = pd.read_csv(f"{Path(os.getcwd()).resolve().parent}/city_count.csv", names=["city", "count"])
data.plot()
plt.show()

In [ ]:
sc.stop()

In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("bus").getOrCreate()

In [21]:
a = spark.read.csv(f"{Path(os.getcwd()).resolve().parent}/city_count.csv", inferSchema=True, header=None)
data_frame = a.createOrReplaceTempView("bus")

In [50]:
qs = """
select 
    *
from 
    bus
where 
    _c0 like '%광역%' or 
    _c0 like '%특별%' 
"""

spark.sql(qs).show()

+----------+-----+
|       _c0|  _c1|
+----------+-----+
|대구광역시| 5068|
|대전광역시| 3005|
|광주광역시| 3099|
|인천광역시| 6401|
|세종특별시| 2684|
|울산광역시| 3714|
|서울특별시|12658|
+----------+-----+

